In [1]:
import datetime as dt
import os
import urllib
from pathlib import Path
import h5py

import matplotlib.pyplot as plt
import numpy as np
import spiceypy as spice
from IPython.display import display, clear_output
from astropy.coordinates import spherical_to_cartesian
from scipy.spatial.distance import cdist
from scipy.spatial.transform import Rotation as R
from surrender.geometry import vec3, quat

import craterdetection.common.constants as const
from craterdetection.common.camera import Camera
from craterdetection.common.conics import generate_mask, crater_representation, ellipse_axes
from craterdetection.common.coordinates import suborbital_coords
from craterdetection.common.spice import get_sol_incidence, get_sun_pos, get_earth_pos
from craterdetection.common.surrender import setup_renderer
from craterdetection.matching.database import extract_robbins_dataset, load_craters

In [2]:
_BASE_URL = 'https://naif.jpl.nasa.gov/pub/naif/'
_KERNEL_ROOT = Path('../data/spice_kernels')
def download_kernel(file_path, base_url=_BASE_URL, base_folder=_KERNEL_ROOT):
    if isinstance(file_path, str):
        file_path = Path(file_path)

    local_path = base_folder / file_path
    url = base_url + file_path.as_posix()

    # Create necessary sub-directories in the DL_PATH direction
    local_path.parent.mkdir(parents=True, exist_ok=True)

    try:
        # If the file is not present in the download directory -> download it
        if not os.path.isfile(local_path):
            print(f"Downloading {url}", end="  ")
            # Download the file with the urllib  package
            urllib.request.urlretrieve(str(url), str(local_path))
            print("Done.")
        else:
            print(f"{base_folder / file_path} already exists!")
    except urllib.error.HTTPError as e:
        print(f"Error: \n{url} could not be found: ", e)

kernels_ = [
               'generic_kernels/lsk/naif0012.tls',
               'generic_kernels/pck/pck00010.tpc',
               'pds/data/lro-l-spice-6-v1.0/lrosp_1000/data/spk/de421.bsp',
               'pds/data/lro-l-spice-6-v1.0/lrosp_1000/data/pck/moon_pa_de421_1900_2050.bpc',
               'pds/data/lro-l-spice-6-v1.0/lrosp_1000/data/fk/moon_assoc_pa.tf',
               'pds/data/lro-l-spice-6-v1.0/lrosp_1000/data/fk/moon_080317.tf'
           ]
kernels = list(map(Path, kernels_))

for k in kernels:
    download_kernel(k)

spice.furnsh(list(map(lambda x: str(_KERNEL_ROOT / x), kernels)))
s = setup_renderer()

..\data\spice_kernels\generic_kernels\lsk\naif0012.tls already exists!
..\data\spice_kernels\generic_kernels\pck\pck00010.tpc already exists!
..\data\spice_kernels\pds\data\lro-l-spice-6-v1.0\lrosp_1000\data\spk\de421.bsp already exists!
..\data\spice_kernels\pds\data\lro-l-spice-6-v1.0\lrosp_1000\data\pck\moon_pa_de421_1900_2050.bpc already exists!
..\data\spice_kernels\pds\data\lro-l-spice-6-v1.0\lrosp_1000\data\fk\moon_assoc_pa.tf already exists!
..\data\spice_kernels\pds\data\lro-l-spice-6-v1.0\lrosp_1000\data\fk\moon_080317.tf already exists!
Surrender Python Client
code revision : c17439eea9ae041bf49364d67a2a232fc89c1faa
error connecting to 127.0.0.1 : Connection refused
error: you must connect to a server before sending commands.


RuntimeError: error: you must connect to a server before sending commands.


In [ ]:
# Constants:
sun_radius = 696342000
earth_radius = 6371e3
ua2km = 149597870.700
ua = ua2km * 1e3
R_moon = 1737.4

s.createBRDF('sun', 'sun.brdf', {})
s.createShape('sun', 'sphere.shp', {'radius': sun_radius})
s.createBody('sun', 'sun', 'sun', [])

s.createBRDF("mate", "mate.brdf", {})
s.createShape("earth_shape", "sphere.shp", {'radius': earth_radius})
s.createBody("earth", "earth_shape", "mate", ["earth.jpg"])

s.createBRDF('hapke', 'hapke.brdf', {})
s.createSphericalDEM('moon', 'FullMoon.dem', 'hapke', 'lroc_color_poles.tiff')
s.setObjectElementBRDF('moon', 'moon', 'hapke')
s.setObjectAttitude('moon', np.array([0, 0, 0, 1]))

s.setObjectPosition('moon', (0, 0, 0))
s.setObjectAttitude('moon', quat(vec3(1, 0, 0), 0))
s.setObjectAttitude('moon', R.from_euler('z', np.pi, degrees=False).as_quat())

In [ ]:
def generate_dataset(size,
                     axis_threshold=(5, 100),
                     resolution=const.CAMERA_RESOLUTION,
                     min_sol_incidence=0,
                     max_sol_incidence=85,
                     filled=False,
                     ellipse_limit=1.4,
                     arc_lims=0.85,
                     diamlims=(1, 200),
                     instancing=False,
                     randomized_orientation=True
                     ):
    lat_cat, long_cat, major_cat, minor_cat, psi_cat, crater_id = extract_robbins_dataset(
        load_craters("../data/lunar_crater_database_robbins_2018.csv", diamlims=diamlims, ellipse_limit=ellipse_limit, arc_lims=arc_lims)
    )
    r_craters_cat = np.array(np.array(spherical_to_cartesian(const.RMOON, lat_cat, long_cat))).T[..., None]
    C_craters_cat = crater_representation(major_cat, minor_cat, psi_cat)

    images_dataset = np.empty((size, 1, *resolution), np.float32)
    if instancing:
        masks_dataset = np.empty((size, 1, *resolution), np.int16)
    else:
        masks_dataset = np.empty((size, 1, *resolution), np.bool_)
    position_dataset = np.empty((size, 3, 1), np.float64)
    attitude_dataset = np.empty((size, 3, 3), np.float64)
    sol_incidence_dataset = np.empty((size, 1), np.float16)
    date_dataset = np.empty((size, 3), int)

    s.setImageSize(*resolution)
    for i in range(size):
        date = dt.date(2021, np.random.randint(1, 12), 1)
        date_dataset[i] = np.array((date.year, date.month, date.day))
        sun_pos, _ = get_sun_pos(date)
        earth_pos, _ = get_earth_pos(date)

        s.setObjectPosition('earth', earth_pos*1e3)
        s.setObjectPosition('sun', sun_pos*1e3)
        sol_incidence = np.inf

        while not (min_sol_incidence <= sol_incidence <= max_sol_incidence):
            lat = np.random.random()*180 - 90
            long = np.random.random()*360 - 180
            cam_alt = np.random.randint(150, 250)
            cam = Camera.from_coordinates(lat,
                                          long,
                                          cam_alt,
                                          convert_to_radians=True,
                                          resolution=resolution
                                          )

            r_ilumin = suborbital_coords(cam.r).ravel()
            sol_incidence = get_sol_incidence(date, r_ilumin)

        # save cam.r, sol_incidence
        position_dataset[i] = cam.r
        sol_incidence_dataset[i] = sol_incidence

        if randomized_orientation:
            # Rotations are incremental (order matters)
            cam.rotate('roll', np.random.random()*360 - 180)
            cam.rotate('pitch', np.random.random()*60 - 30)
            cam.rotate('yaw', np.random.random()*60 - 30)

        # save cam.T
        attitude_dataset[i] = cam.T

        s.setObjectPosition('camera', cam.r.ravel()*1e3)
        s.setObjectAttitude('camera', R.from_matrix(cam.T).as_quat())

        visible = (cdist(r_craters_cat.squeeze(), cam.r.T) <= np.sqrt(2 * cam_alt * const.RMOON + cam_alt**2)).ravel()
        r_craters = r_craters_cat[visible]
        C_craters = C_craters_cat[visible]

        r_craters_img = cam.project_crater_centers(r_craters)
        in_image = np.logical_and.reduce(np.logical_and(r_craters_img > -20, r_craters_img < cam.resolution[0]+20), axis=1)

        r_craters = r_craters[in_image]
        C_craters = C_craters[in_image]

        A_craters = cam.project_crater_conics(C_craters, r_craters)

        a_proj, b_proj = ellipse_axes(A_craters)
        axis_filter = np.logical_and(a_proj >= axis_threshold[0], b_proj >= axis_threshold[0])
        axis_filter = np.logical_and(axis_filter, np.logical_and(a_proj <= axis_threshold[1], b_proj <= axis_threshold[1]))
        A_craters = A_craters[axis_filter]
        mask = generate_mask(A_craters, cam.resolution, filled=filled, instancing=instancing)

        masks_dataset[i] = mask[None, None, ...]

        s.render()
        image = s.getImageGray32F()

        images_dataset[i] = image[None, None, ...]
        if 100*((i+1) / size) % 10 == 0:
            print(' ... ', end='')
            print(f"[{(i+1) / size:.0%}]", end='')
    print()
    return images_dataset, masks_dataset, position_dataset, attitude_dataset, date_dataset, sol_incidence_dataset


In [ ]:
ellipse_limit = 1.1
arc_lims = 0.3
diamlims = (2, 150)
axis_threshold = (4, 40)
min_sol_incidence = 20
max_sol_incidence = 60
resolution = (256, 256)
training_dset_size = 10000
validation_dset_size = 1000
test_dset_size = 1000
filled = True
instancing = True
randomized_orientation = True

In [ ]:
n_demo = 20
images, mask, _, _, _, _ = generate_dataset(n_demo,
                                                        min_sol_incidence=min_sol_incidence,
                                                        resolution=resolution,
                                                        axis_threshold=axis_threshold,
                                                        ellipse_limit=ellipse_limit,
                                                        filled=filled,
                                                        instancing=instancing,
                                                        arc_lims=arc_lims,
                                                        randomized_orientation=randomized_orientation
                                                        )
fig, axes = plt.subplots(n_demo, 2, figsize=(10, 5*n_demo))
for i in range(n_demo):
    axes[i, 0].imshow(images[i, 0], cmap='Greys_r')
    axes[i, 1].imshow(mask[i, 0], cmap='Greys_r')

In [ ]:
with h5py.File("../data/dataset_instanced.h5", 'w') as hf:
    g_header = hf.create_group("header")
    g_header.create_dataset('ellipse_limit', data=ellipse_limit)
    g_header.create_dataset('arc_lims', data=arc_lims)
    g_header.create_dataset('diam_lims', data=diamlims)
    g_header.create_dataset('axis_lims', data=axis_threshold)
    g_header.create_dataset('min_sol_incidence', data=min_sol_incidence)
    g_header.create_dataset('resolution', data=resolution)

    for group_name, dset_size in zip(
            ("training", "validation", "test"),
            (training_dset_size, validation_dset_size, test_dset_size)
        ):
        print(f"Creating dataset '{group_name}' @ {dset_size} images")
        group = hf.create_group(group_name)

        (
            images,
            masks,
            position,
            attitude,
            date,
            sol_incidence
        ) = generate_dataset(dset_size,
                             min_sol_incidence=min_sol_incidence,
                             resolution=resolution,
                             axis_threshold=axis_threshold,
                             ellipse_limit=ellipse_limit,
                             filled=filled,
                             instancing=instancing,
                             arc_lims=arc_lims,
                             randomized_orientation=randomized_orientation)
        for ds, name in zip(
                (images, masks, position, attitude, date, sol_incidence),
                ("images", "masks", "position", "attitude", "date", "sol_incidence")
            ):

            group.create_dataset(name, data=ds)
